# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f9d00ad8a90>
├── 'a' --> tensor([[ 0.7730,  0.7115, -2.5610],
│                   [-1.4018,  0.2748,  0.4407]])
└── 'x' --> <FastTreeValue 0x7f9d00ad8bb0>
    └── 'c' --> tensor([[ 1.4024, -0.7648,  1.7645,  0.2820],
                        [ 0.0420, -1.4195,  0.5089, -1.2967],
                        [ 0.3106,  1.4347, -1.7090,  1.5024]])

In [4]:
t.a

tensor([[ 0.7730,  0.7115, -2.5610],
        [-1.4018,  0.2748,  0.4407]])

In [5]:
%timeit t.a

84.2 ns ± 0.594 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f9d00ad8a90>
├── 'a' --> tensor([[-1.2691,  0.9885, -1.7208],
│                   [ 1.4440, -1.8461,  1.3594]])
└── 'x' --> <FastTreeValue 0x7f9d00ad8bb0>
    └── 'c' --> tensor([[ 1.4024, -0.7648,  1.7645,  0.2820],
                        [ 0.0420, -1.4195,  0.5089, -1.2967],
                        [ 0.3106,  1.4347, -1.7090,  1.5024]])

In [7]:
%timeit t.a = new_value

90.3 ns ± 0.941 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.7730,  0.7115, -2.5610],
               [-1.4018,  0.2748,  0.4407]]),
    x: Batch(
           c: tensor([[ 1.4024, -0.7648,  1.7645,  0.2820],
                      [ 0.0420, -1.4195,  0.5089, -1.2967],
                      [ 0.3106,  1.4347, -1.7090,  1.5024]]),
       ),
)

In [10]:
b.a

tensor([[ 0.7730,  0.7115, -2.5610],
        [-1.4018,  0.2748,  0.4407]])

In [11]:
%timeit b.a

77.8 ns ± 0.543 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.0511,  0.4410,  1.8480],
               [ 0.6830, -1.4493, -2.4825]]),
    x: Batch(
           c: tensor([[ 1.4024, -0.7648,  1.7645,  0.2820],
                      [ 0.0420, -1.4195,  0.5089, -1.2967],
                      [ 0.3106,  1.4347, -1.7090,  1.5024]]),
       ),
)

In [13]:
%timeit b.a = new_value

664 ns ± 7.28 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.12 µs ± 7.35 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

14.4 µs ± 272 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

278 µs ± 6.46 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

272 µs ± 6.47 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f9c548026d0>
├── 'a' --> tensor([[[ 0.7730,  0.7115, -2.5610],
│                    [-1.4018,  0.2748,  0.4407]],
│           
│                   [[ 0.7730,  0.7115, -2.5610],
│                    [-1.4018,  0.2748,  0.4407]],
│           
│                   [[ 0.7730,  0.7115, -2.5610],
│                    [-1.4018,  0.2748,  0.4407]],
│           
│                   [[ 0.7730,  0.7115, -2.5610],
│                    [-1.4018,  0.2748,  0.4407]],
│           
│                   [[ 0.7730,  0.7115, -2.5610],
│                    [-1.4018,  0.2748,  0.4407]],
│           
│                   [[ 0.7730,  0.7115, -2.5610],
│                    [-1.4018,  0.2748,  0.4407]],
│           
│                   [[ 0.7730,  0.7115, -2.5610],
│                    [-1.4018,  0.2748,  0.4407]],
│           
│                   [[ 0.7730,  0.7115, -2.5610],
│                    [-1.4018,  0.2748,  0.4407]]])
└── 'x' --> <FastTreeValue 0x7f9c54802f40>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

49 µs ± 396 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f9c54802d30>
├── 'a' --> tensor([[ 0.7730,  0.7115, -2.5610],
│                   [-1.4018,  0.2748,  0.4407],
│                   [ 0.7730,  0.7115, -2.5610],
│                   [-1.4018,  0.2748,  0.4407],
│                   [ 0.7730,  0.7115, -2.5610],
│                   [-1.4018,  0.2748,  0.4407],
│                   [ 0.7730,  0.7115, -2.5610],
│                   [-1.4018,  0.2748,  0.4407],
│                   [ 0.7730,  0.7115, -2.5610],
│                   [-1.4018,  0.2748,  0.4407],
│                   [ 0.7730,  0.7115, -2.5610],
│                   [-1.4018,  0.2748,  0.4407],
│                   [ 0.7730,  0.7115, -2.5610],
│                   [-1.4018,  0.2748,  0.4407],
│                   [ 0.7730,  0.7115, -2.5610],
│                   [-1.4018,  0.2748,  0.4407]])
└── 'x' --> <FastTreeValue 0x7f9c54802310>
    └── 'c' --> tensor([[ 1.4024, -0.7648,  1.7645,  0.2820],
                        [ 0.0420, -1.4195,  0.5089, -1.2967],
                 

In [23]:
%timeit t_cat(trees)

45.3 µs ± 449 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

86.3 µs ± 1.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.7730,  0.7115, -2.5610],
                [-1.4018,  0.2748,  0.4407]],
       
               [[ 0.7730,  0.7115, -2.5610],
                [-1.4018,  0.2748,  0.4407]],
       
               [[ 0.7730,  0.7115, -2.5610],
                [-1.4018,  0.2748,  0.4407]],
       
               [[ 0.7730,  0.7115, -2.5610],
                [-1.4018,  0.2748,  0.4407]],
       
               [[ 0.7730,  0.7115, -2.5610],
                [-1.4018,  0.2748,  0.4407]],
       
               [[ 0.7730,  0.7115, -2.5610],
                [-1.4018,  0.2748,  0.4407]],
       
               [[ 0.7730,  0.7115, -2.5610],
                [-1.4018,  0.2748,  0.4407]],
       
               [[ 0.7730,  0.7115, -2.5610],
                [-1.4018,  0.2748,  0.4407]]]),
    x: Batch(
           c: tensor([[[ 1.4024, -0.7648,  1.7645,  0.2820],
                       [ 0.0420, -1.4195,  0.5089, -1.2967],
                       [ 0.3106,  1.4347, -1.7090,  1.5024]],
         

In [26]:
%timeit Batch.stack(batches)

110 µs ± 869 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.7730,  0.7115, -2.5610],
               [-1.4018,  0.2748,  0.4407],
               [ 0.7730,  0.7115, -2.5610],
               [-1.4018,  0.2748,  0.4407],
               [ 0.7730,  0.7115, -2.5610],
               [-1.4018,  0.2748,  0.4407],
               [ 0.7730,  0.7115, -2.5610],
               [-1.4018,  0.2748,  0.4407],
               [ 0.7730,  0.7115, -2.5610],
               [-1.4018,  0.2748,  0.4407],
               [ 0.7730,  0.7115, -2.5610],
               [-1.4018,  0.2748,  0.4407],
               [ 0.7730,  0.7115, -2.5610],
               [-1.4018,  0.2748,  0.4407],
               [ 0.7730,  0.7115, -2.5610],
               [-1.4018,  0.2748,  0.4407]]),
    x: Batch(
           c: tensor([[ 1.4024, -0.7648,  1.7645,  0.2820],
                      [ 0.0420, -1.4195,  0.5089, -1.2967],
                      [ 0.3106,  1.4347, -1.7090,  1.5024],
                      [ 1.4024, -0.7648,  1.7645,  0.2820],
                      [ 0.0420, -

In [28]:
%timeit Batch.cat(batches)

195 µs ± 3.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

537 µs ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
